<a href="https://colab.research.google.com/github/TayasanM/Fake_Image_Detection_TPOT/blob/main/TPOT_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=be66b84595cfb7b9a46d17d85cabc58057549e4fb21e25bde47116c46b62a7f0
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [5]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, RocCurveDisplay, auc
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import timeit

[link text](https://)# Load the Data

In [7]:
data = pd.read_csv('CIFAKE_test_Shuffel.csv')
#,CIFAKE_Shuffel

data.head()

,Unnamed: 0,Imnagenum,Outlabel,meanValues,varianceValues,homogeneityValues,contrastValues,dissimilarityValues,entropyValues,secondMomentValues,...,energyValues,stdDeviation,skewnessValue,kurtosisValue,histogramValues,areaValues,perimeterValues,roughnessValues,averageGrayLevels,overlappingRatios
0,0,18711,0,99.750606,2156.161389,0.948374,0.103252,0,7.438871,2156.119545,...,0.154293,46.434485,0.157198,2.170917,-2182,51529,2307,2.866928,99.750606,4.461369
1,1,11553,0,89.543888,611.017097,0.972436,0.055127,0,6.619042,611.005240,...,0.304753,24.718760,0.002454,2.729043,12088,51529,2188,2.719046,89.543888,0.494048
2,2,9199,1,155.820742,4550.598494,0.928361,0.144401,0,7.681828,4550.510182,...,0.125700,67.458124,-0.763956,2.693772,-150,51529,2163,2.687978,155.820742,6.018047
3,3,14844,0,113.877778,2731.753110,0.968205,0.063589,0,7.303287,2731.700096,...,0.178514,52.266176,0.171078,1.714877,693,51529,1490,1.851635,113.877778,4.644971
4,4,15874,0,120.394050,4546.355454,0.965122,0.069756,0,7.198981,4546.267225,...,0.189309,67.426667,0.585479,1.715890,571,51529,848,1.053817,120.394050,12.493069


In [8]:
X = data.drop(["Imnagenum","dissimilarityValues"],axis=1,inplace=True)
#X = data.drop(["Imnagenum",		"homogeneityValues",	"contrastValues",	"dissimilarityValues",	"entropyValues",	"secondMomentValues",	"correlationValues",	"energyValues",	"stdDeviation",	"skewnessValue","histogramValues",	"kurtosisValue"],axis=1,inplace=True)
#X = data.drop(["Imnagenum","homogeneityValues","entropyValues",	"secondMomentValues","correlationValues",	"energyValues",	"stdDeviation",	"skewnessValue",	"kurtosisValue"	,"contrastValues","dissimilarityValues","areaValues","roughnessValues","perimeterValues",	"averageGrayLevels","histogramValues",	"overlappingRatios"],axis=1,inplace=True)
X = data.drop('Outlabel', axis=1)
y = data['Outlabel']


In [9]:
X.head()

,Unnamed: 0,meanValues,varianceValues,homogeneityValues,contrastValues,entropyValues,secondMomentValues,correlationValues,energyValues,stdDeviation,skewnessValue,kurtosisValue,histogramValues,areaValues,perimeterValues,roughnessValues,averageGrayLevels,overlappingRatios
0,0,99.750606,2156.161389,0.948374,0.103252,7.438871,2156.119545,0.976497,0.154293,46.434485,0.157198,2.170917,-2182,51529,2307,2.866928,99.750606,4.461369
1,1,89.543888,611.017097,0.972436,0.055127,6.619042,611.005240,0.959574,0.304753,24.718760,0.002454,2.729043,12088,51529,2188,2.719046,89.543888,0.494048
2,2,155.820742,4550.598494,0.928361,0.144401,7.681828,4550.510182,0.983543,0.125700,67.458124,-0.763956,2.693772,-150,51529,2163,2.687978,155.820742,6.018047
3,3,113.877778,2731.753110,0.968205,0.063589,7.303287,2731.700096,0.988922,0.178514,52.266176,0.171078,1.714877,693,51529,1490,1.851635,113.877778,4.644971
4,4,120.394050,4546.355454,0.965122,0.069756,7.198981,4546.267225,0.992344,0.189309,67.426667,0.585479,1.715890,571,51529,848,1.053817,120.394050,12.493069


In [10]:
y.head()

0    0
1    0
2    1
3    0
4    0
Name: Outlabel, dtype: int64

### TPOT Settings with generations=10 and population_size=100

In [ ]:
%%time
# to calculate time


# splitting the data
# training = 80%
# test = 20%
X_train, X_test, y_train, y_test = train_test_split(X.astype(np.float64), y.astype(np.float64), train_size=0.8, test_size=0.2, random_state=30)

# TPOT classifer
tpot = TPOTClassifier(generations=10, population_size=100, verbosity=2,cv=5, random_state=42)
#pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                  #  random_state=42, verbosity=2)
#,scoring='f1'

# fitting the classifier
tpot.fit(X_train, y_train)

# displaying the accuracy
print(tpot.score(X_test, y_test))

# saving the pipeline
tpot.export('tpot_images_pipeline_gen_10_pop_100.py')

Optimization Progress:   0%|          | 0/1100 [00:00<?, ?pipeline/s]

### Classifier (Suggested by TPOT)